## Settings

In [1]:
# -*- coding: utf8 -*-
# coding: utf8
# import packages
import pandas as pd
import numpy as np
import timeit
import datetime
import os
import StringIO
from docx import Document
from docx.shared import Inches
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_ALIGN_PARAGRAPH
import pymongo
from pymongo import MongoClient
import re
import splunklib.client as client
import splunklib.results as results
from collections import OrderedDict
os.chdir('function directory')
from autoGenEventDoc_funclist_mongo import *

# DECLARE PARAMETERS
EVENT_ID = 'event_id' # change here
db_name = 'your mongoDB collection name' # change here
year = EVENT_ID[0:2]
month = EVENT_ID[2:4]
day = EVENT_ID[4:6]
MongoDB_ip = 'YOUR MONGODB SERVER IP' # change here

if db_name == 'iii':
    HOST = "YOUR SPLUNK SERVER IP" # change here
elif db_name == 'ites':
    HOST = "YOUR SPLUNK SERVER IP" # change here
PORT = "YOUR SPLUNK SERVER PORT" # change here
USERNAME = 'YOUR SPLUNK SERVER USERNAME' # change here
PASSWORD = 'YOUR SPLUNK SERVER PASSWORD' # change here

## MongoDB Part

In [2]:
# query all necessary event information from mongodb
# connect to the mongoDB server
client = MongoClient(MongoDB_ip, 'mongoDB port')
# select the collection
db = client.monitor_report
# query data via specific information
# data_mongo = get_dataByDateMonthDay(db, year, month, day)
data_mongo = get_dataByEventID(db, EVENT_ID, db_name)
# preview the query data
data_mongo.head()

MongoDB Data has successfully exported to dataframe from MongoDB via get_dataByEventID!



,_id,advice,date,dst_host_name,dst_ip,dst_port,end_time,event_description,event_id,event_name,org,src_host_name,src_ip,src_port,start_time
0,5a52d8a97d6a9462030819aa,資安監控室長期趨勢追蹤發現創研所主機異常連線至可疑中繼站與DN，Malware DN建議先透...,2017-10-02,"blog.qooza.hk, stun.iptel.org, drive.google-in...",,53,2017-10-02T23:59:59.999,該事件為創研所屬網段電腦連線至ICST所列管的中繼站主機，有疑似惡意程式活動之行為特徵。,171002-1,INDICATOR-COMPROMISE Suspicious .pw/.tk/.ml dn...,民生科服大樓(ITES),,140.92.140.170 (IDEAS),,2017-10-02T00:00:00.000


In [3]:
# fetch the source ip information for next step
ip_data = get_srcIP_info(data_mongo.src_ip)
print ip_data

['140.92.140.170']


## Splunk Data Part

In [4]:
# import splunk packages
from autoGenEventDoc_funclist_splunk import *
# query all necessary data from splunk
# column name infomation
# get time range
timeRange = {"earliest_time": str(data_mongo.start_time.values[0]),
                  "latest_time": str(data_mongo.end_time.values[0])}
# get ip address
search_ip = data_mongo.src_ip
# connect to the splunk server
service = Connect2Server(HOST, PORT, USERNAME, PASSWORD)
# get the event diary data
data_splunk = queryData4(service, ip_data[0], timeRange, db_name)
# preview the query data
print data_splunk

You have successfully connected to the splunk server!

查詢期間觸發事件一覽 search has been successfully done!

查詢期間攻擊事件趨勢 search has been successfully done!

事件日誌 search has been successfully done!

=== All necessary data have been already exported!!! ===

[                          _time  \
0   2017-10-02 00:00:00.000 CST   
1   2017-10-02 01:00:00.000 CST   
2   2017-10-02 02:00:00.000 CST   
3   2017-10-02 03:00:00.000 CST   
4   2017-10-02 04:00:00.000 CST   
5   2017-10-02 05:00:00.000 CST   
6   2017-10-02 06:00:00.000 CST   
7   2017-10-02 07:00:00.000 CST   
8   2017-10-02 08:00:00.000 CST   
9   2017-10-02 09:00:00.000 CST   
10  2017-10-02 10:00:00.000 CST   
11  2017-10-02 11:00:00.000 CST   
12  2017-10-02 12:00:00.000 CST   
13  2017-10-02 13:00:00.000 CST   
14  2017-10-02 14:00:00.000 CST   
15  2017-10-02 15:00:00.000 CST   
16  2017-10-02 16:00:00.000 CST   
17  2017-10-02 17:00:00.000 CST   
18  2017-10-02 18:00:00.000 CST   
19  2017-10-02 19:00:00.000 CST   
20  2017-10-02 2

## Generate Docx Part

In [5]:
# setup all colname of the document
colName_1 = [['問題單', '問題單開立時間'], '事件名稱', '事件描述', 
             ['來源主機名稱', '來源IP位置', '來源通訊埠','目的主機名稱', '目的主機位置', '目的通訊埠', '事件描述'],
             '建議措施', ['參考資料', '附件資料'],'如果您對此通告的內容有疑問或有關於此事件的建議，請勿直接回覆此信件，請"\n以下述聯絡資訊與我們聯絡。\n資安科技研究所 技術研發中心 (http://ctti.iii.org.tw/)\n資策會資安所資安監控室(SOC)\n地 址： 臺北市松山區民生東路四段133 號14 樓\n聯絡電話： 02-66078979\n傳真電話： 02-66078998\n電子郵件信箱： chhsieh@iii.org.tw"',
             '附件', ['事件趨勢圖', 'RawData']]
colName_1 = decodeList_2_UTF8(colName_1)

In [6]:
quesSheet_info = [str(data_mongo.event_id.values[0]), year+month+day]
quesSheet_info = decodeList_2_UTF8(quesSheet_info)
eventNameContents = str(data_mongo.event_name.values[0])
eventNameContents = eventNameContents.decode("utf-8")
ed = data_mongo.event_description.values[0]
ed = ed.encode('utf-8')
eventDesContents = [str(data_mongo.src_host_name.values[0]), str(data_mongo.src_ip.values[0]) , 
                    str(data_mongo.src_port.values[0]), str(data_mongo.dst_host_name.values[0]),
                    str(data_mongo.dst_ip.values[0]), str(data_mongo.dst_port.values[0]), ed]
eventDesContents = decodeList_2_UTF8(eventDesContents)
sc = data_mongo.advice.values[0]
suggestContents = sc

In [7]:
# create document
document = Document()
font = document.styles['Normal'].font
font.name = 'STKaitiTC-Regular'

# creat table object
table = document.add_table(rows=14, cols=4)
table.style = 'TableGrid'
table.alignment = WD_TABLE_ALIGNMENT.CENTER

# row1
titleText = colName_1[0]
title_row = table.rows[0].cells
title_row[0].text = titleText[0]
title_row[1].text = quesSheet_info[0]
title_row[2].text = titleText[1]
title_row[3].text = quesSheet_info[1]

# row2
eventNameCell = table.rows[1].cells
eventName = colName_1[1]
eventNameCell[0].text = eventName
eventNameContentCell = cellMerge(table, [1, 1], [1, 4])
eventNameContentCell.text = eventNameContents

# row3
eventDesCell = table.rows[2].cells
eventDesTitle = rowMerge(table, [2, 0], [7, 0])
eventDesTitle.text = colName_1[2]
for i in xrange(7):
    cell_des = cellMerge(table, [i+2, 1], [i+2, 2])
    cell_des.text = colName_1[3][i]
    cell_con = cellMerge(table, [i+2, 2], [i+2, 4])
    cell_con.text = eventDesContents[i]

# row4
suggestDo = table.rows[8].cells
suggestDo[0].text = colName_1[4]
suggestDes = cellMerge(table, [8, 2], [8, 4])
suggestDes.text = suggestContents

# row5
referInfo = table.rows[9].cells
referInfo[0].text = colName_1[5][0]
referText = cellMerge(table, [9, 1], [9, 4])
referText.text = colName_1[5][1]

# row6
contactInfo = table.rows[10].cells
contactInfo = cellMerge(table, [10, 0], [10, 4])
contactInfo.text = colName_1[6]

# row7
attachWord = table.rows[11].cells
attachWord = cellMerge(table, [11, 0], [11, 4])
attachWord.text = colName_1[7]

# row8
appText = table.rows[12].cells
appText = cellMerge(table, [12, 0], [12, 4])
appText.text = colName_1[8][0]

# row9
appContents = table.rows[13].cells
appContents = cellMerge(table, [13, 0], [13, 4])
rawData = ''
for i in range(len(data_splunk[-1])):
    rawData = rawData + data_splunk[-1][i]
appContents.text = colName_1[8][1] + '\n' + rawData


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/docx/styles/styles.py:54: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  warn(msg, UserWarning)


In [8]:
# document.add_page_break()
event_id = str(data_mongo.event_id.to_string()[5:])
if db_name == 'iii':
    docxFileName = 'III事件通知單' + event_id + '.docx'
elif db_name == 'ites':
    docxFileName = 'ITES事件通知單' + event_id + '.docx'
document.save(docxFileName)